In [1]:
#install modules
# import sys
# !{sys.executable} -m pip install xgboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import matplotlib
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

/home/dushan/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/dushan/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/dushan/venv/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
######################_getting_data_from_files_######################
# training data
train_file_path = 'train.csv'
train_data = pd.read_csv(train_file_path)
print("training data size: ",len(train_data))
      
# test data
predicting_data_file_path='test.csv'
predicting_data = pd.read_csv(predicting_data_file_path)
print("predicting data size: ",len(predicting_data))

training data size:  59055
predicting data size:  25310


In [4]:
train_data.dtypes

pickup_lat         float64
pickup_long        float64
drop_lat           float64
drop_long          float64
pickup_time         object
taxi_model          object
travel_distance    float64
trip_duration      float64
dtype: object

In [5]:
######################_selecting_affecting_columns_##################

# any variable updated (or created) after the target value is realized should be excluded to prevent Leaky Predictors

# to get the size of data
# train_data.shape
all_cols=train_data.columns 

# if selecting all columns drop predicting column
predicting_cols=all_cols.drop(['trip_duration']) 

# predicting_cols=['LotArea', 'YearBuilt','1stFlrSF','FullBath','BedroomAbvGr','TotRmsAbvGrd']
#####################################################################

In [6]:
######################_X_y_predictingX_##############################
X=train_data[predicting_cols]
y=train_data.trip_duration

predicting_X_=predicting_data[predicting_cols]
#####################################################################

In [7]:
######################_convert_column_data_like_time################



# date into hours
def get_hour(x):
    h=datetime.datetime.strptime(x.split('.')[0], '%Y-%m-%d %H:%M:%S').time()
    return h.hour

# date into day
def get_day(x):
    d=datetime.datetime.strptime(x.split('.')[0], '%Y-%m-%d %H:%M:%S').date().weekday()
    return d

X['pickup_time_hour'] = X['pickup_time'].apply(lambda x: get_hour(x))

X['pickup_time_day'] = X['pickup_time'].apply(lambda x: get_day(x))

del X['pickup_time']


predicting_X_['pickup_time_hour'] = predicting_X_['pickup_time'].apply(lambda x: get_hour(x))

predicting_X_['pickup_time_day'] = predicting_X_['pickup_time'].apply(lambda x: get_day(x))

del predicting_X_['pickup_time']
# after adding new column pickup_time_hour delete pickup_time column


#####################################################################

In [8]:
######################_handling_categorial_##########################
# dtypes of colmns
# print(X.dtypes)
# print(len(X))

# # handling categorial cols using one_hot_encoding
# one_hot_encoded_X = pd.get_dummies(X)
# one_hot_encoded_predicting_X = pd.get_dummies(predicting_X_)
# X_encoded, predicting_X_encoded = one_hot_encoded_X.align(one_hot_encoded_predicting_X, join='left', axis=1)

#####################################################################

# # handling categorial cols using label LabelEncoder
X_encoded=X
predicting_X_encoded=predicting_X_

label_encoder_taxi = LabelEncoder()
label_encoder_time = LabelEncoder()
X_encoded['taxi_model']=label_encoder_taxi.fit_transform(X.taxi_model.values)
# X_encoded['pickup_time']=label_encoder_time.fit_transform(X.pickup_time.values)
# print(X_encoded.head())

predicting_X_encoded['taxi_model']=label_encoder_taxi.transform(predicting_X_.taxi_model.values)
# predicting_X_encoded['pickup_time']=label_encoder_time.transform(predicting_X_.pickup_time.values)
# print(predicting_X_encoded.head())

#####################################################################

In [9]:
# Create table for missing data analysis
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

draw_missing_data_table(predicting_X_encoded)

,Total,Percent
pickup_time_day,0,0.0
pickup_time_hour,0,0.0
travel_distance,0,0.0
taxi_model,0,0.0
drop_long,0,0.0
drop_lat,0,0.0
pickup_long,0,0.0
pickup_lat,0,0.0


In [10]:
######################_null_col_handling_############################
# #no use if using pipeline
# handling null columns
# checking null cols #train_data.isnull().any()


# droping
# cols_with_missing = [col for col in train_data.columns if train_data[col].isnull().any()]
# reduced_trin_data = train_data.drop(cols_with_missing, axis=1)
# reduced_test_data = test_data.drop(cols_with_missing, axis=1)

# using SimpleImputer

# for training data
imputed_X = X_encoded.copy()

# for predicting data
imputed_predicting_X=predicting_X_encoded.copy()

# cols with null in training data
cols_with_missing_training_data = (col for col in X_encoded.columns if X_encoded[col].isnull().any())

# cols with null in predicting data
cols_with_missing_predicting_data=(col for col in predicting_X_encoded.columns if predicting_X_encoded[col].isnull().any())

# set of all null cols
cols_with_missing=list(set(list(cols_with_missing_training_data)+list(cols_with_missing_predicting_data)))

for col in cols_with_missing:
    imputed_X_train[col + '_was_missing'] = imputed_X_train[col].isnull()
    imputed_X_test[col + '_was_missing'] = imputed_X_test[col].isnull()
    imputed_predicting_X[col + '_was_missing'] = imputed_predicting_X[col].isnull()

imputer = Imputer()
# imputering data in training data
imputed_X_train = imputer.fit_transform(imputed_X) #fit imputer and transform data

# imputering data in predicting data
imputed_predicting_X = imputer.transform(imputed_predicting_X)
#####################################################################

In [39]:
# ######################_deviding_data_X_y_############################
# deviding training data for checking correctness
train_X,test_X,train_y,test_y=train_test_split(imputed_X_train,y,random_state=0)

#required data for prediction
predicting_X=imputed_predicting_X
#####################################################################

In [40]:
######################_model_selection_##############################
def get_mae_model(model_,X, y):
	# model=DecisionTreeRegressor()
    model=model_
    mae_val= -1 * cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error').mean()
    print("MAE with ",type(model).__name__," \t:",mae_val)
    return mae_val

# finding max leaf nodes
def get_MAE_nodes(max_leaf_nodes, training_X, predicting_X, training_y, predicting_values_y):
    model=DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes,random_state=0)
    model.fit(training_X,training_y)
    predicted_val_y=model.predict(predicting_X)
    MAE=mean_absolute_error(predicting_values_y,predicted_val_y)
    return (MAE)
# for max_leaf_nodes in [5,50,60,65,67,70,75,80,85,90]:
#     current_MAE=get_MAE_nodes(max_leaf_nodes,train_X,val_X,train_y,val_y)
#     print("max leaf nodes: %d \t\t Mean Absolute Error: %d" %(max_leaf_nodes,current_MAE))

# get_mae_model(DecisionTreeRegressor(),imputed_X_train,train_y)
# get_mae_model(RandomForestRegressor(),imputed_X_train,train_y)
# get_mae_model(XGBRegressor(),imputed_X_train,train_y)
# get_mae_model(GradientBoostingRegressor(),imputed_X_train,train_y)

#####################################################################

In [53]:
######################_trainging_the_model_##########################
# find the number of nodes for least MAE and create the model according to it
# model=DecisionTreeRegressor(max_leaf_nodes=75,random_state=0)  #model with specifying max leaf nodes
# model.fit(train_X, train_y)

# model= RandomForestRegressor()
# model.fit(train_X, train_y)

# model = GradientBoostingRegressor()
# model.fit(train_X, train_y)

# model=XGBRegressor()
# scpecify n_jobs for XGBRegressor if dataset is too large. assign num of cores in machine to n_jobs 
# find using early_stopping_rounds and assign it to n_estimators, start with a big number
model=XGBRegressor(n_estimators=2000,learning_rate=0.05)
model.fit(train_X,train_y,early_stopping_rounds=20, eval_set=[(test_X, test_y)], verbose=False)
#####################################################################

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
######################_using_pipeline_###############################
# pipeline = make_pipeline(Imputer(), RandomForestRegressor())
# pipeline.fit(train_X, train_y)
# predictions = pipeline.predict(test_X)
#####################################################################

In [55]:

######################_plot_partial_dependence_######################
# # should be an instance of BaseGradientBoosting 
# # cols_to_use=['travel_distance', 'pickup_time_hour','trip_duration']
# cols_to_use=all_cols
# # feature is defining which should be plotted from cols_to_use
# y_plots = plot_partial_dependence(model, features=[1,2], X=train_X, feature_names=cols_to_use,grid_resolution=10)
# matplotlib.pyplot.show(block=True)
#####################################################################

In [16]:
######################_cross_validation_#############################
# # use cross_val_score instead of train_test_split for small size data sets
# # this is not needed for large data sets. train_test_split is faster
# imputer_=Imputer()
# pipeline = make_pipeline(imputer_, XGBRegressor())
# pipeline.fit(X_encoded, y)

# # imputing predicting test using same imputer
# imputed_predicting_X = imputer_.transform(predicting_X)

# # possible methods for scoring =>'accuracy' , 'neg_mean_absolute_error'
# scores = cross_val_score(pipeline, X_encoded, y, scoring='neg_mean_absolute_error')
# print(scores)
#####################################################################

In [54]:
######################_evaluating_the_model_#########################
predicted_values_for_traininig_set=model.predict(test_X)
# predicted_values_for_traininig_set=pipeline.predict(imputed_X_test)

# print correct and predicted values
# print("actual",'predicted\n')
# for idx,val in enumerate(test_y):
# 	print (val,predicted_values_for_traininig_set[idx])

# print mean absolute error
print("\nMAE:\t",mean_absolute_error(test_y,predicted_values_for_traininig_set))
#####################################################################


MAE:	 8.348123096250577


In [18]:
######################_prediction_for_required_data_#################
predicted_result=model.predict(predicting_X)
# predicted_result=pipeline.predict(imputed_predicting_X)
#####################################################################

In [19]:
######################_visualization_of_prediction_##################
# creating a csv
# submission = pd.DataFrame({'Id': predicting_data.Id, 'SalePrice': predicted_result})
# submission.to_csv('submission.csv', index=False)

# creating txt file
np.savetxt('output.txt',predicted_result,fmt='%.2f')

#####################################################################
#                                                                   #
#                                END                                #
#                                                                   #
#####################################################################

In [20]:


############                    ############
############   plots examples   ############
############                    ############



In [21]:
# # Histograms allow you to plot the distributions of numeric variables.

# # HistogramPython
# # Distribution Plot (a.k.a. Histogram)
# sns.distplot(train_data.travel_distance)

In [22]:
# # distribution plots  according to values on one field
# g = sns.FacetGrid(train_data, col='taxi_model')
# g = (g.map(sns.distplot,"travel_distance", hist=True))

In [23]:
# # reg line plot with scatter

# sns.lmplot(x='travel_distance', y='trip_duration', data=train_data,scatter=False,hue='taxi_model')
# sns.lmplot(x='travel_distance', y='trip_duration', data=train_data, fit_reg=False,hue='taxi_model')

In [24]:
# pair plot
# https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166

# sns.pairplot(train_data)
#here diagonal is showing the distribution of that variable

# sns.pairplot(train_data, hue = 'taxi_model')

# Create a pair plot colored by taxi_model with a density plot of the # diagonal and format the scatter plots.
# sns.pairplot(train_data, hue = 'taxi_model', 
#              vars = ['travel_distance', 'trip_duration'],
#              diag_kind = 'kde',
#              plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
#              size = 4)

# can add constraints
# sns.pairplot(train_data[train_data['year'] >= 2000],
#              vars = ['travel_distance', 'trip_duration'], 
#              hue = 'continent', diag_kind = 'kde', 
#              plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
#              size = 4);

 

In [25]:
# # Create an instance of the PairGrid class.

# grid = sns.PairGrid(data= train_data,
#                     vars = ['travel_distance', 'trip_duration']
#                     , size = 4)
# # Map a scatter plot to the upper triangle
# grid = grid.map_upper(plt.scatter)

# # Map a histogram to the diagonal
# grid = grid.map_diag(plt.hist, bins = 10, color = 'darkred', 
#                      edgecolor = 'k')

# # Map a density plot to the lower triangle
# grid = grid.map_lower(sns.kdeplot, cmap = 'Reds')

In [26]:
# # plot heatmap

# f,ax = plt.subplots(figsize=(8, 8))

# # Calculate correlations
# corr = train_data.corr()
 
# # Heatmap
# sns.heatmap(corr,annot=True,linewidths=3, fmt= '.2f',ax=ax)



In [27]:
# # plot heatmap only down side

# f,ax = plt.subplots(figsize=(6, 6))
# mask = np.zeros_like(corr)
# mask[np.triu_indices_from(mask)] = True
# with sns.axes_style("white"):
#     ax_ = sns.heatmap(corr, mask=mask,linewidths=3,annot=True,ax=ax)

In [28]:
# # plot result

# x=[1,3,5,6,7]
# y=[2,10,20,3,5]
# plt.plot(x,y,'bo',x,y,'r--')